TO DO LIST : 
 - Ajout du tracé ---> OK
 - Ajout de data augmentation --->A DEBUGGUER
 - Ajout trace des callBacks dans DataFrame, pour savoir si un callback s'est enclenché (par urgent)
 - Ajout de prediction sur les modèles ayant eu un bon score en validation

-  AJOUTER CETTE VARIABLE
"""from tensorflow.keras.optimizers import Adam 
    adam = Adam(learning_rate=0.001)"""    
           Chercher comment éviter l'affichage des WARNINGS.


## Import des librairies et fonctions

In [1]:
import joblib
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import time
import os
from joblib import load,dump
import numpy as np

#Keras
import keras
from keras.applications.vgg16 import VGG16

# tensorlflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,Dropout, Flatten, Dense
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback,ModelCheckpoint,EarlyStopping,ReduceLROnPlateau


## Fonctions secondaires

In [2]:
# fonction pour récupérer le chemin absolu d'un fichier placé au même endroit que le notebook en cours
def get_output_path_file(output_filename):
    notebook_filename = os.path.abspath("__file__")
    notebook_directory = os.path.dirname(notebook_filename)
    return os.path.join(notebook_directory, output_filename)

# fonction pour récupérer le chemin absolu d'un dossier placé au même endroit que le notebook en cours
def get_output_path_folder(output_foldername):
    notebook_filename = os.path.abspath("__file__")
    notebook_directory = os.path.dirname(notebook_filename)
    return os.path.join(notebook_directory, output_foldername)

#fonction de generation des deux dataset train et validation
def generation_train_validation(repertoire_train,image_size,batch_size,validation_split):
    
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(     #création d'un itérateur qui va charger les images 
        repertoire_train,                                                     #lors des étapes d'entrainements
        labels='inferred', 
        label_mode='categorical', 
        seed=123,  color_mode='rgb', 
        image_size=image_size ,
        batch_size=batch_size,
        validation_split=validation_split,
        subset='training')

    validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        repertoire_train,
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=batch_size,
        image_size=image_size,
        validation_split=validation_split,
        subset='validation',  
        seed=123)
  
    return train_dataset, validation_dataset

def generation_test(repertoire_test,image_size,batch_size):
    test_dataset = tf.keras.preprocessing.image_dataset_from_directory(       
        repertoire_test,
        labels='inferred',
        label_mode='categorical', 
        seed=123,  color_mode='rgb', #Images à 3 canaux
        image_size=image_size,
        batch_size=batch_size)
    
    return test_dataset

# fonction de récupération des noms et shape de chaque couche du model
def get_layer_info(model):
    layer_info = []
    for layer in model.layers:
        layer_info.append({
            "Layer Name": layer.name#,
            #"Output Shape": layer.output_shape
        })
    return layer_info

#Fonction pour tracer la courbe de suivi de loss et accuracy au fil des epochs
def tracer_courbe_suivi(nom_df_de_suivi,numero_ligne_à_tracer):
    
    chemin_df = get_output_path_file(nom_df_de_suivi)
    df = load(chemin_df)
   
    #Courbe d'entrainement
    
    epochs_range = range(1,len(df.loss[numero_ligne_à_tracer])+1,1)
                         
    loss = df.loss[numero_ligne_à_tracer]
    accuracy = df.acc[numero_ligne_à_tracer]
    val_loss = df.val_loss[numero_ligne_à_tracer]
    val_accuracy = df.val_acc[numero_ligne_à_tracer]
    
    plt.figure(figsize = (8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, accuracy, label='Training Accuracy')
    plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.xticks(range(1,len(df.loss[numero_ligne_à_tracer])+1,1))
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.xticks(range(1,len(df.loss[numero_ligne_à_tracer])+1,1))
    plt.title('Training and Validation Loss')
    
    return plt.show()


## Fonction principale

In [3]:
#Définition des CallBacks
early_stopping = EarlyStopping(monitor="val_loss",
                               min_delta = 0.01,
                               patience = 3,
                               verbose=1,
                               restore_best_weights = True)

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss",
                                         patience = 2,
                                         factor = 0.1,
                                         cooldown = 4,
                                         verbose=1,
                                         min_lr = 1e-10)

model_checkpoint = ModelCheckpoint(filepath = get_output_path_folder("Fichiers_points_de_controles"),
                                    monitor = "val_loss",
                                    verbose = 1,
                                    save_best_only = False,
                                    save_weights_only= False,
                                    moder = "auto",
                                    save_freq="epoch")


#Fonction de suivi de modèles de DL
def test_DL_predict(repertoire_train,repertoire_test,image_size,batch_size,validation_split,liste_modele,optimizer,loss,
                    metrics,epochs,chemin_df_score_DL,nom_fichier_controle):
      
    train_dataset,validation_dataset = generation_train_validation(repertoire_train,
                                                                   image_size,batch_size,
                                                                   validation_split) 
    test_dataset = generation_test(repertoire_test,image_size,batch_size)  #Ajout repertoire test

    #nombre total d'échantillons dans l'ensemble de données d'entraînement
    nb_img_train = tf.data.experimental.cardinality(train_dataset).numpy() # FL :On récupère une valeur de 1821 d'où le faible nombre d'étapes

    #nombre total d'échantillons dans l'ensemble de données de validation
    nb_img_test = tf.data.experimental.cardinality(validation_dataset).numpy()
    
    model_layers_info = []
    for model in liste_modele:
        layers_info = get_layer_info(model)
        model_layers_info.append(layers_info)
    if os.path.exists(chemin_df_score_DL):
        df_import = load(chemin_df_score_DL)
        print("récupération du df existant")
    else:
        df_import = pd.DataFrame(columns=["image_size","batch_size","epochs", "optimizer","fonction loss", "loss","acc", "val_loss","val_acc", "Duree_entrainement en sec","test_acc","Nom_fichier_contrôle"])
        print("création d'un dataframe")
    score = []
    for i, model in enumerate(liste_modele):
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        print(f"debut de l'entrainement du modèle {i}")
        debut = time.time()
        # entrainement avec la méthode .fit()
        history = model.fit(train_dataset, 
                        epochs = epochs,
                        callbacks=[early_stopping, reduce_learning_rate,model_checkpoint],
                        steps_per_epoch = nb_img_train//batch_size,
                        validation_data = validation_dataset,
                        validation_steps = nb_img_test//batch_size)
        
        print(f"fin de l'entrainement du modèle {i}")
        fin = time.time()
        duree_entrainement = fin - debut
        layer_info = model_layers_info[i]
        
        # doc : https://www.tensorflow.org/api_docs/python/tf/keras/Model
        
        if np.max(history.history['val_acc']) >  0.4 :
            score_test = model.evaluate(test_dataset,batch_size = batch_size, verbose='auto')
            print("%s: %.2f%%" % (model.metrics_names[0], score_test[0]*100))
            print("%s: %.2f%%" % (model.metrics_names[1], score_test[1]*100))

        else:
            score_test  = 'Non calculé'
            print("le score de test n'a pas été calculé car le score d'entrainement est trop faible")
        
        model.save(nom_fichier_controle + '.h5')
        # Loads the weights
               
        model_scores = {
            "image_size":f"{image_size}",
            "batch_size": f"{batch_size}",
            "model":f"{layer_info}",
            "epochs": f"{epochs}",
            "optimizer": f"{optimizer}",
            "fonction loss": f"{loss}",
            "loss": history.history['loss'],
            "acc": history.history['acc'],
            "val_loss":history.history['val_loss'],
            "val_acc":history.history['val_acc'],
            "Duree_entrainement en sec": duree_entrainement,
            "test_acc": score_test,
            "Nom_fichier_contrôle": f"{nom_fichier_controle}.h5"
            
        }
            
        score.append(model_scores)
    df_score = pd.DataFrame(score)
    df = pd.concat([df_import, df_score], ignore_index=True)
    return df

